In [2]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")
OPENAI_MODEL_NAME = os.environ["OPENAI_MODEL_NAME"]
OPENAI_BASE_URL = os.environ["OPENAI_BASE_URL"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
EMBEDDING_MODEL = os.environ["EMBEDDING_MODEL"]


llm = ChatOpenAI(temperature=0, model=OPENAI_MODEL_NAME, api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

In [3]:
llm.invoke('去年中秋节是哪天？').content

'去年（2022年）的中秋节是在阳历的9月10日。中秋节是中国的传统节日，通常在农历的八月十五日庆祝，这一天月亮最圆，象征着团圆和丰收。'

In [3]:
functions = [{
        'name': 'get_item_info',
        'description': '获取时间范围内某个产品的信息',
        'parameters': {
            'type': 'object',
            'properties': {
                'item': {
                    'type': 'string',
                    'description':
                    '产品名称',
                },
                'start': {
                    'type': 'string',
                    'description':
                    '时间范围的起始时间'
                },
                'end': {
                    'type': 'string',
                    'description':
                    '时间范围的结束时间'
                },
            },
            'required': ['event', 'start', 'end'],
        }
    }]

In [4]:
get_item_info = llm.bind_tools(functions, tool_choice="get_item_info")

In [5]:
get_item_info.invoke('2023年1月12至2月1日笔记本的销量')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-b7305d344f624fdd883851036a0752bf', 'function': {'arguments': '{ "item": "笔记本", "start": "2023年1月12日", "end": "2023年2月1日" }', 'name': 'get_item_info'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 36, 'total_tokens': 74}, 'model_name': 'qwen2-7b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3694b26f-198c-4d54-8127-4add446b8af6-0', tool_calls=[{'name': 'get_item_info', 'args': {'item': '笔记本', 'start': '2023年1月12日', 'end': '2023年2月1日'}, 'id': 'chatcmpl-tool-b7305d344f624fdd883851036a0752bf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 36, 'output_tokens': 38, 'total_tokens': 74})

In [6]:
get_item_info.invoke('去年九月份笔记本的销量')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-1d2746e9ab2c4ef58e0f19d1f2d77d64', 'function': {'arguments': '{ "item": "笔记本", "start": "去年九月份", "end": "去年九月份" }', 'name': 'get_item_info'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 25, 'total_tokens': 50}, 'model_name': 'qwen2-7b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e1300b6d-746f-4f3b-a1ce-888a7119e316-0', tool_calls=[{'name': 'get_item_info', 'args': {'item': '笔记本', 'start': '去年九月份', 'end': '去年九月份'}, 'id': 'chatcmpl-tool-1d2746e9ab2c4ef58e0f19d1f2d77d64', 'type': 'tool_call'}], usage_metadata={'input_tokens': 25, 'output_tokens': 25, 'total_tokens': 50})

In [7]:
# %pip install jionlp

import jionlp
query = '去年双11的营收多少'
def date_extract(query):
    date = jionlp.parse_time(query)
    return date['time']
date = date_extract(query)
print(date)

# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.
# jiojio - Successfully load C funcs for CWS and POS acceleration.
['2023-11-11 00:00:00', '2023-11-11 23:59:59']


In [8]:
date_modify_prompt = '''
你是一个日期转换助手，请按照如下格式对用户输入进行转化。

用户输入：
2023年端午节的订单销量是多少

日期：
['2023-06-22 00:00:00', '2023-06-22 23:59:59']

输出：
2023-06-22 00:00:00到2023-06-22 23:59:59的订单销量是多少

用户输入：
2022年中秋节的营收多少

日期：
['2022-09-10 00:00:00', '2022-09-10 23:59:59']

输出：
2022-09-10 00:00:00到2022-09-10 23:59:59营收多少

用户输入：
{}

日期：
{}

输出：
'''

In [9]:
query = '去年第一季度的销量怎么样'
modify_query = llm.invoke(date_modify_prompt.format(query, date_extract(query))).content
print(modify_query)

2023-01-01 00:00:00到2023-03-31 23:59:59的销量怎么样
